In [74]:
from scipy.optimize import linprog
import numpy as np
import math


# Hackaton

C -> Electricity Cost

72 weights ( represent the cost of each hour per mode)
- w00, w01, w02, w03, w04, w05, w06, w07, ...

modes (hours_mode_off, hours_mode_eco, hours_mode_confort)
- h_00_off, h_00_eco, h_00_conf, h_01_off, h_01_eco, h_01_conf, ...

this modes are binary variables (0 or 1)


Expression
- C = w00*h_00_off + w01*h_00_eco + w02*h_00_conf + w03*h_01_off + w04*h_01_eco + w05*h_01_conf + w06*h_02_off + w07*h_02_eco + w08*h_02_conf + ...


Contraints

Binary Variables and Confort score at least 124
- h_00_off + h_00_eco + h_00_conf = 1
- h_01_off + h_01_eco + h_01_conf = 1
...
 - 124 <= (h_00_eco + h_01_eco + h_02_eco + ... + h_23_eco)*4 +(h_00_conf + h_01_conf + ... + h_23_conf)*8

In [88]:
#weights represent the cost of each hour per mode
#need to be calculated through the api data
weights_temp = [0.1, 0.3, 0.6]*12 + [0.2, 0.5, 1.2]*12 

In [89]:
obj = weights_temp


lhs_eq = [[1, 1, 1]+[0]*69, [0]*3+[1]*3+[0]*66, [0]*6+[1]*3+[0]*63, [0]*9+[1]*3+[0]*60, [0]*12+[1]*3+[0]*57, [0]*15+[1]*3+[0]*54, [0]*18+[1]*3+[0]*51, [0]*21+[1]*3+[0]*48, [0]*24+[1]*3+[0]*45, [0]*27+[1]*3+[0]*42, [0]*30+[1]*3+[0]*39, [0]*33+[1]*3+[0]*36, [0]*36+[1]*3+[0]*33, [0]*39+[1]*3+[0]*30, [0]*42+[1]*3+[0]*27, [0]*45+[1]*3+[0]*24, [0]*48+[1]*3+[0]*21, [0]*51+[1]*3+[0]*18, [0]*54+[1]*3+[0]*15, [0]*57+[1]*3+[0]*12, [0]*60+[1]*3+[0]*9, [0]*63+[1]*3+[0]*6, [0]*66+[1]*3+[0]*3, [0]*69+[1]*3]
rhs_eq = [1]*24

lhs_ineq = [[0, -4, -8, 0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8,0, -4, -8]]
rhs_ineq = [-124]


In [90]:
bnd = [(0,1) for i in range(72)]
integrality = [1 for i in range(72)]

In [91]:
def OptimizeCost(Weights, contraints, bounds, integrality):
    """inputs:
    Weights: list of weights for each hour
    contraints: list of contraints
    bounds: list of bounds
    integrality: 1 : Integer variable; decision variable must be an integer within bounds."""

    opt = linprog(c=Weights, A_ub=contraints['lhs_ineq'], b_ub=contraints['rhs_ineq'],
              A_eq=contraints['lhs_eq'], b_eq=contraints['rhs_eq'], bounds=bounds, integrality=integrality,
              method="highs")

    #find if its eco, off or conf and conver to string 
    policy = np.array(opt['x']).reshape(24,3).argmax(axis=1)
    #convert array number to off, eco or confort
    policy = np.vectorize(lambda x: 'off' if x == 0 else 'eco' if x == 1 else 'conf')(policy)
    
    cost = opt['fun']
    score = np.array(124)+opt['ineqlin']['residual'][0]
    return policy, cost, score


In [92]:
results = OptimizeCost(weights_temp, {'lhs_eq': lhs_eq, 'rhs_eq': rhs_eq, 'lhs_ineq': lhs_ineq, 'rhs_ineq': rhs_ineq}, bnd, integrality)

In [93]:
print("Policy: ", results[0])
print("Electricity Cost: ", results[1])
print("Confort Score: ", results[2])

Policy:  ['conf' 'conf' 'eco' 'conf' 'conf' 'conf' 'conf' 'eco' 'conf' 'conf'
 'conf' 'conf' 'eco' 'eco' 'eco' 'eco' 'eco' 'off' 'eco' 'eco' 'eco' 'eco'
 'off' 'off']
Electricity Cost:  11.7
Confort Score:  124.0


In [96]:
temperature_array = [10]*6 + [20]*12 + [15]*6

In [94]:
def calculateWeights(temperature_array, energy_cost_array):
    #Temperature array is the temperature for each hour of the day
    #Energy cost array is the cost of energy for each hour of the day

    #calculate the weights for each hour
    weights = []
    modes = ['off', 'eco', 'conf']
    for i in range(len(temperature_array)):
        for mode in modes:
            external_temp = temperature_array[i]
            if mode == 'off':
                #1KW/hour regardless of temperature
                weights.append(energy_cost_array[i]*1)
            elif mode == 'eco':
                if external_temp < 10:
                    #1.6KW/hour
                    weights.append(energy_cost_array[i]*1.6)
                elif external_temp >= 10 and external_temp <= 20:
                    #0.8KW/hour
                    weights.append(energy_cost_array[i]*0.8)
                elif external_temp > 20:
                    #0.4KW/hour
                    weights.append(energy_cost_array[i]*0.4)
            elif mode == 'conf':

                if external_temp < 0:
                    #2.4KW/hour+9KW/hour
                    weights.append(energy_cost_array[i]*(2.4+9))
                elif external_temp < 10:
                    #2.4KW/hour
                    weights.append(energy_cost_array[i]*2.4)
                elif external_temp >= 10 and external_temp <= 20:
                    #1.6KW/hour
                    weights.append(energy_cost_array[i]*1.6)
                elif external_temp > 20:
                    #0.8KW/hour
                    weights.append(energy_cost_array[i]*0.8)
    return weights
    
        
